# Data Quality

**This notebook is used to showcase the quality of the found in the structured database tables, after the data has been processed by the data engineering applications.**

## Installation of Dependencies

In [1]:
!pip install psycopg2-binary

## Importing Dependencies

In [2]:
import psycopg2

## Establish Connection to the Database

In [3]:
connection = psycopg2.connect(
    database = "capstone_project",
    user = "udacity",
    password = "udacity",
    host = "172.28.1.2",
    port = "5432"
)

cursor = connection.cursor()

## Execute Data Quality Queries

### Find all the Partition Tables

In [4]:
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_name LIKE 'packets_%' ORDER BY table_name;")
packets_partitions = cursor.fetchall()

cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_name LIKE 'originate_packets_%' ORDER BY table_name;")
originate_packets_partitions = cursor.fetchall()

cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_name LIKE 'response_packets_%' ORDER BY table_name;")
response_packets_partitions = cursor.fetchall()

print("Packets Partitions:\tOriginate Packets Partitions:\tResponse Packets Partitions:")
print("------------------------------------------------------------------------------------")
for packets, originate_packets, response_packets in zip(packets_partitions, originate_packets_partitions, response_packets_partitions):
    print(f"{packets[0]}\t{originate_packets[0]}\t{response_packets[0]}")

Packets Partitions:	Originate Packets Partitions:	Response Packets Partitions:
------------------------------------------------------------------------------------
packets_2018_10_2	originate_packets_2018_10_2	response_packets_2018_10_2
packets_2018_10_3	originate_packets_2018_10_3	response_packets_2018_10_3
packets_2018_10_4	originate_packets_2018_10_4	response_packets_2018_10_4
packets_2018_12_20	originate_packets_2018_12_20	response_packets_2018_12_20
packets_2018_12_21	originate_packets_2018_12_21	response_packets_2018_12_21
packets_2018_12_22	originate_packets_2018_12_22	response_packets_2018_12_22
packets_2018_5_10	originate_packets_2018_5_10	response_packets_2018_5_10
packets_2018_5_11	originate_packets_2018_5_11	response_packets_2018_5_11
packets_2018_5_12	originate_packets_2018_5_12	response_packets_2018_5_12
packets_2018_5_13	originate_packets_2018_5_13	response_packets_2018_5_13
packets_2018_5_14	originate_packets_2018_5_14	response_packets_2018_5_14
packets_2018_5_19	origin

### Confirm the Packet Count

In [5]:
cursor.execute("SELECT count(*) FROM packets;")
packets_count = cursor.fetchone()
print(f"Rows in the Packets table: {packets_count[0]}")

cursor.execute("SELECT count(*) FROM originate_packets;")
originate_packets_count = cursor.fetchone()
print(f"Rows in the Originate Packets table: {originate_packets_count[0]}")

cursor.execute("SELECT count(*) FROM response_packets;")
response_packets_count = cursor.fetchone()
print(f"Rows in the Response Packets table: {response_packets_count[0]}")

print(f"\nIs the total row count the same over all the packet tables? {packets_count[0] == originate_packets_count[0] == response_packets_count[0]}")

Rows in the Packets table: 325307990
Rows in the Originate Packets table: 325307990
Rows in the Response Packets table: 325307990

Is the total row count the same over all the packet tables? True


### Perform JOIN Aggregation on the Data

In [6]:
cursor.execute("""
    SELECT * FROM packets
    JOIN originate_packets ON packets.uid = originate_packets.uid
    JOIN response_packets ON packets.uid = response_packets.uid
    JOIN asn ON packets.response_network_id = asn.network_id
    JOIN city_blocks ON packets.response_network_id = city_blocks.network_id
    JOIN city_locations ON city_blocks.geoname_id = city_locations.geoname_id
    LIMIT 1
""")

join_select = cursor.fetchall()
print(join_select)

[('1532535870', 'C8sXPJZSWuGQUhTVf', '192.168.100', '195.189.68', 'tcp', '-', '-', 'S0', 0, 'S', '(empty)', 'Malicious', 'PartOfAHorizontalPortScan', datetime.date(2018, 7, 25), 'C8sXPJZSWuGQUhTVf', '192.168.100.111', 24671, '-', '-', 1, 40, datetime.date(2018, 7, 25), 'C8sXPJZSWuGQUhTVf', '195.189.68.85', 81, '-', '-', 0, 0, datetime.date(2018, 7, 25), '195.189.68.0/23', 41007, 'Ctc Astana Ltd', '195.189.68', '195.189.68.0/23', 1526273, 1522867, None, 0, 0, None, 51.1879, 71.4357, 1000, '195.189.68', 1526273, 'en', 'AS', 'Asia', 'KZ', 'Kazakhstan', 'AST', 'Nur-Sultan', None, None, 'Nur-Sultan', None, 'Asia/Almaty', 0)]


## Close the Connection

In [7]:
connection.close()
cursor.close()